In [1]:
# from google.colab import drive
# drive.mount('/content/drive/', force_remount=True)

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from pathlib import Path
from sklearn.preprocessing import StandardScaler

In [3]:
# Need to upload "boardgames.csv" to files every session.
file_path = ('boardgames.csv')
boardgame_df = pd.read_csv(file_path)
# Unicode Problem on row 24
boardgame_df.head(5)

,objectid,name,average,avgweight,boardgamecategory,boardgamemechanic
0,174430,Gloomhaven,8.85292,3.8078,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative..."
1,161936,Pandemic Legacy Season 1,8.62499,2.8301,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma..."
2,167791,Terraforming Mars,8.42299,3.2313,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma..."
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:..."
4,224517,Brass Birmingham,8.62031,3.9122,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market..."


In [4]:
# Our decoding
decode_lambda = lambda x: bytearray(x, 'utf-8').decode('unicode-escape')
# Applying the decoding to the column, ignore errors.
boardgame_df['name'] = boardgame_df['name'].apply(lambda x: decode_lambda(x))
# Unicode replaced with proper characters
boardgame_df.head(5)

C:\Users\annyl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: invalid escape sequence '\/'
  
C:\Users\annyl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: invalid escape sequence '\T'
  
C:\Users\annyl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: invalid escape sequence '\A'
  
C:\Users\annyl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: invalid escape sequence '\W'
  
C:\Users\annyl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: invalid escape sequence '\O'
  
C:\Users\annyl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: invalid escape sequence '\D'
  
C:\Users\annyl\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: invalid escape sequence '\I'
  


,objectid,name,average,avgweight,boardgamecategory,boardgamemechanic
0,174430,Gloomhaven,8.85292,3.8078,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative..."
1,161936,Pandemic Legacy Season 1,8.62499,2.8301,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma..."
2,167791,Terraforming Mars,8.42299,3.2313,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma..."
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:..."
4,224517,Brass Birmingham,8.62031,3.9122,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market..."


In [5]:
boardgame_df = boardgame_df.rename(columns={'objectid': 'ID', 'name': 'Name', 'average': 'Avg Rating', 'avgweight': 'Complexity', 'boardgamecategory': 'Category', 'boardgamemechanic': 'Mechanic'})

# Other languages are supported by the decoding: Row 70
boardgame_df.head(5)

,ID,Name,Avg Rating,Complexity,Category,Mechanic
0,174430,Gloomhaven,8.85292,3.8078,"['Adventure', 'Exploration', 'Fantasy', 'Fight...","['Campaign / Battle Card Driven', 'Cooperative..."
1,161936,Pandemic Legacy Season 1,8.62499,2.8301,"['Environmental', 'Medical']","['Action Points', 'Cooperative Game', 'Hand Ma..."
2,167791,Terraforming Mars,8.42299,3.2313,"['Economic', 'Environmental', 'Industry / Manu...","['Card Drafting', 'End Game Bonuses', 'Hand Ma..."
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850,"['Card Game', 'Civilization', 'Economic']","['Action Points', 'Auction/Bidding', 'Auction:..."
4,224517,Brass Birmingham,8.62031,3.9122,"['Economic', 'Industry / Manufacturing', 'Tran...","['Hand Management', 'Income', 'Loans', 'Market..."


In [6]:
boardgame_trunc = boardgame_df.drop(["Category","Mechanic"],axis=1)
boardgame_trunc.head()

,ID,Name,Avg Rating,Complexity
0,174430,Gloomhaven,8.85292,3.8078
1,161936,Pandemic Legacy Season 1,8.62499,2.8301
2,167791,Terraforming Mars,8.42299,3.2313
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850
4,224517,Brass Birmingham,8.62031,3.9122


In [7]:
categories_df = boardgame_df[['ID', 'Category']].copy()
categories_df['Category'] = categories_df['Category'].apply(lambda x: x[1:len(x)-1].split(', '))

mechanics_df = boardgame_df[['ID', 'Mechanic']].copy()
mechanics_df['Mechanic'] = mechanics_df['Mechanic'].apply(lambda x: x[1:len(x)-1].split(', '))

In [8]:
pd.set_option('display.max_rows', 500)

cat_counts = categories_df.explode('Category')
cat_counts.columns.str.replace("'","")
cat_vc = cat_counts.groupby("Category").size().sort_values(ascending=False)

categories_to_replace = cat_vc[cat_vc < 50].index

# # Replace in dataframe
for cats in categories_to_replace:
    cat_counts['Category'] = cat_counts['Category'].replace(cats,"Other")

# # Check to make sure binning was successful
cat_vc_check = cat_counts.groupby("Category").size().sort_values(ascending=False)

cat_vc_check

Category
'Card Game'                     5686
'Wargame'                       3500
'Fantasy'                       2361
'Party Game'                    1723
'Dice'                          1712
'Fighting'                      1565
'Abstract Strategy'             1541
'Childrens Game'                1521
'Science Fiction'               1495
'Economic'                      1423
'Bluffing'                      1136
'World War II'                  1129
'Animals'                       1107
'Humor'                         1064
'Deduction'                     1046
'Adventure'                     1017
'Action / Dexterity'             999
'Movies / TV / Radio theme'      980
'Miniatures'                     925
'Medieval'                       888
'Exploration'                    792
'Racing'                         709
'Ancient'                        697
'Negotiation'                    636
'Real-time'                      623
'Horror'                         593
'Nautical'                   

In [9]:
categories_to_replace

Index([''Pike and Shot'', ''American Indian Wars'', ''Game System'',
       ''Korean War'', ''KOSMOS'', ''Schmidt Spiele'', ''White Goblin Games'',
       ''Ravensburger Spieleverlag GmbH'', ''Piatnik'', ''Blue Orange (EU)'',
       ...
       ''Gale Force Nine'', ''GDM Games'', ''G&RRR'', ''Front Porch Classics'',
       ''Freie Spiel'', ''Floodgate Games'', ''Si-yat-sa'',
       ''Fiery Dragon Productions'', ''Feuerland Spiele'',
       ''u685cu904au5eb5 (Ouyuuan)''],
      dtype='object', name='Category', length=288)

In [27]:
cat_dummies = pd.get_dummies(cat_counts)
cat_final = cat_dummies.groupby("ID").sum()
cat_final

,Category_'Abstract Strategy',Category_'Action / Dexterity',Category_'Adventure',Category_'Age of Reason',Category_'American Civil War',Category_'American Revolutionary War',Category_'American West',Category_'Ancient',Category_'Animals',Category_'Arabian',...,Category_'Travel',Category_'Trivia',Category_'Video Game Theme',Category_'Vietnam War',Category_'Wargame',Category_'Word Game',Category_'World War I',Category_'World War II',Category_'Zombies',Category_Other
ID,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292961,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
292962,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
294612,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
pd.set_option('display.max_rows', 500)

mech_counts = mechanics_df.explode('Mechanic')
mech_vc = mech_counts.groupby("Mechanic").size().sort_values(ascending=False)

mech_types_to_replace = mech_vc[mech_vc < 175].index

# # Replace in dataframe
for mechs in mech_types_to_replace:
    mech_counts['Mechanic'] = mech_counts['Mechanic'].replace(mechs,"Other")

# # Check to make sure binning was successful
mech_vc_check = mech_counts.groupby("Mechanic").size().sort_values(ascending=False)

mech_dummies = pd.get_dummies(mech_counts)
mech_final = mech_dummies.groupby("ID").sum()

In [35]:
dfs_to_merge = [boardgame_trunc, cat_final, mech_final]

boardgame_inter = pd.merge(boardgame_trunc, cat_final, on="ID", how='outer')
boardgame_final = pd.merge(boardgame_inter, mech_final, on="ID", how='outer')

In [36]:
boardgame_final

,ID,Name,Avg Rating,Complexity,Category_'Abstract Strategy',Category_'Action / Dexterity',Category_'Adventure',Category_'Age of Reason',Category_'American Civil War',Category_'American Revolutionary War',...,Mechanic_'Take That',Mechanic_'Team-Based Game',Mechanic_'Tile Placement',Mechanic_'Trading',Mechanic_'Trick-taking',Mechanic_'Variable Phase Order',Mechanic_'Variable Player Powers',Mechanic_'Voting',Mechanic_'Worker Placement',Mechanic_Other
0,174430,Gloomhaven,8.85292,3.8078,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,161936,Pandemic Legacy Season 1,8.62499,2.8301,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,167791,Terraforming Mars,8.42299,3.2313,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,2
3,182028,Through the Ages A New Story of Civilization,8.49419,4.3850,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,224517,Brass Birmingham,8.62031,3.9122,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,5154,Franchise,6.00000,0.0000,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
19996,5158,Punto y Raya,4.00000,0.0000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19997,5159,3 Up,6.00000,0.0000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
19998,5160,Dino-Checkers,5.77500,0.0000,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
